# Linpackベンチマーク -- HPL

---
OpenHPC環境で、Linpackベンチマークの[HPL](http://www.netlib.org/benchmark/hpl/)を実行します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。

* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる

> この Notebookでは、管理ユーザ以外が実行した状況に合わせるために Ansible ではなく ssh で操作を行います。

マスターノードに対して SSH でログインできることを確認します。マスターノードのアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address =

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'
# user = 'user00'

user =

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

## 準備

HPLを実行するための準備作業を行います。

[HPL](http://www.netlib.org/benchmark/hpl/)のソースを取得してビルドします。
まず、ソースファイルを取得して、ホームディレクトリに展開します。

In [ ]:
hpl_version = '2.3'
hpl_url = f'http://www.netlib.org/benchmark/hpl/hpl-{hpl_version}.tar.gz'

!ssh {ssh_opts} {target} curl -O {hpl_url}
!ssh {ssh_opts} {target} tar xzf hpl-{hpl_version}.tar.gz

後で参照しやすいように、シンボリックリンクを作成します。

In [ ]:
!ssh {ssh_opts} {target} ln -sf hpl-{hpl_version} hpl

HPLのコンパイルオプションを指定します。以下のようなオプションが指定できます。

```
    -DHPL_COPY_L
       force the copy of the panel L before bcast;

    -DHPL_CALL_CBLAS
       call the cblas interface;

    -DHPL_CALL_VSIPL
       call the vsip  library;

    -DHPL_DETAILED_TIMING
       enables detail timers;
```

In [ ]:
hpl_opts = '-DHPL_CALL_CBLAS'

HPL をビルドするときの `arch` 名を指定します。ここで指定した名前を使って後で`Make.{arch}` ファイルを作成します。

In [ ]:
hpl_arch= 'Linux_OPENBLAS'

`Make.{arch}` ファイルを作成します。

In [ ]:
!ssh {ssh_opts} {target} '''cd hpl && module load openblas && sed \
    -e 's%@ARCH@%{hpl_arch}%' \
    -e 's%@LADIR@%\\\$\\\(OPENBLAS_DIR\\\)%' \
    -e 's%@LAINC@%-I\\\$\\\(OPENBLAS_INC\\\)%' \
    -e 's%@LALIB@%-Wl,-rpath,\\\$\\\(OPENBLAS_LIB\\\)\\\ -L\\\$\\\(OPENBLAS_LIB\\\)\\\ -lopenblas%' \
    -e 's%@SHELL@%/bin/sh%' \
    -e 's%@CD@%cd%' \
    -e 's%@CP@%cp%' \
    -e 's%@LN_S@%ln\\\ -s%' \
    -e 's%@MKDIR@%mkdir%' \
    -e 's%@RM@%/bin/rm\\\ -f%' \
    -e 's%@TOUCH@%touch%' \
    -e 's%@CC@%mpicc%' \
    -e 's%@CCNOOPT@%%' \
    -e 's%@CCFLAGS@%-fomit-frame-pointer\\\ -O3\\\ -funroll-loops\\\ -W\\\ -Wall%' \
    -e 's%@LINKER@%mpif77%' \
    -e 's%@LINKFLAGS@%\\\$\\\(CCFLAGS\\\)%' \
    -e 's%@ARCHIVER@%ar%' \
    -e 's%@ARFLAGS@%r%' \
    -e 's%@RANLIB@%echo%' \
    -e 's%@MPDIR@%%' \
    -e 's%@MPINC@%%' \
    -e 's%@MPLIB@%%' \
    -e 's%@F2CDEFS@%%' \
    -e 's%^HPL_OPTS.*%HPL_OPTS\\\ =\\\ {hpl_opts.replace(" ", "\\\\\\ ")}%' \
    setup/Make.UNKNOWN.in > Make.{hpl_arch}'''

作成した `Make.{arch}`の内容を確認します。

In [ ]:
!ssh {ssh_opts} {target} 'cat hpl/Make.{hpl_arch}'

HPLのビルドを行います。

In [ ]:
!ssh {ssh_opts} {target} 'cd hpl && module load openblas && \
    make arch={hpl_arch}'

`xhpl`コマンドがビルドできたことを確認します。

In [ ]:
!ssh {ssh_opts} {target} 'ls -l hpl/bin/{hpl_arch}'

## HPLの実行

HPLを実行します。

### パラメータの設定

計算ノードの数を指定してください。

In [ ]:
# (例)
# num_nodes = 2

num_nodes =

MPIのタスク数を指定してください。

In [ ]:
# (例)
# num_tasks = 4

num_tasks =

### HPL.dat の作成

HPL.dat を作成します。

次のセルの `N`や`NB`の値を必要に応じて書き換えて実行してください。HPL.dat の記述方法については[HPL Tuning](http://www.netlib.org/benchmark/hpl/tuning.html) を参照してください。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
%run scripts/hpl_utils.py


p, q = hpl_pq(num_tasks)
with TemporaryDirectory() as workdir:
    hpl_dat = Path(workdir) / 'HPL.dat'
    with hpl_dat.open(mode='w') as f:
        f.write(f'''HPLinpack benchmark input file
Innovative Computing Laboratory, University of Tennessee
HPL.out      output file name (if any)
6            device out (6=stdout,7=stderr,file)
1            # of problems sizes (N)
10000        Ns
1            # of NBs
192          NBs
0            PMAP process mapping (0=Row-,1=Column-major)
1            # of process grids (P x Q)
{p}            Ps
{q}            Qs
16.0         threshold
1            # of panel fact
2            PFACTs (0=left, 1=Crout, 2=Right)
1            # of recursive stopping criterium
2            NBMINs (>= 1)
1            # of panels in recursion
2            NDIVs
1            # of recursive panel fact.
1            RFACTs (0=left, 1=Crout, 2=Right)
1            # of broadcast
0            BCASTs (0=1rg,1=1rM,2=2rg,3=2rM,4=Lng,5=LnM)
1            # of lookahead depth
0            DEPTHs (>=0)
0            SWAP (0=bin-exch,1=long,2=mix)
1            swapping threshold
1            L1 in (0=transposed,1=no-transposed) form
1            U  in (0=transposed,1=no-transposed) form
0            Equilibration (0=no,1=yes)
8            memory alignment in double (> 0)
''')
    !cat {hpl_dat}
    !scp {ssh_opts} {str(hpl_dat)} {target}:hpl/bin/{hpl_arch}

### ジョブスクリプトの作成

HPLを実行するジョブスクリプトを作成します。

In [ ]:
with TemporaryDirectory() as workdir:
    hpl_batch = Path(workdir) / 'hpl.job'
    with hpl_batch.open(mode='w') as f:
        f.write(f'''#!/bin/bash

#SBATCH -J hpl                # Job name
#SBATCH -o hpl.%j.out         # Name of stdout output file (%j expands to jobId)
#SBATCH -N {num_nodes}                  # Total number of nodes requested
#SBATCH -n {num_tasks}                  # Total number of mpi tasks requested
#SBATCH -t 01:00:00           # Run time (hh:mm:ss)

# Launch MPI-based executable
module load openblas

cd hpl/bin/{hpl_arch}

prun ./xhpl        
''')
    !cat {hpl_batch}
    !scp {ssh_opts} {str(hpl_batch)} {target}:.

### ジョブの実行

HPLのジョブを実行します。

In [ ]:
!ssh {ssh_opts} {target} sbatch hpl.job

ジョブの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

結果を確認します。

> ジョブの実行が完了するまで数分程度かかります。しばらく待ってから次のセルを実行してください。

In [ ]:
!ssh {ssh_opts} {target} 'cat hpl.*.out'